<a href="https://colab.research.google.com/github/Thamarai-Selvam/Itunes-Podcast-DB-India/blob/master/Podcasts_data_miner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#!pip install pattern3

import requests
from bs4 import BeautifulSoup
import pattern3.web as web
import re
import numpy as np
import pandas as pd 


site = 'https://podcasts.apple.com/in/genre/podcasts/id26'

def getlinks(site):
    try:
        url = url = web.URL(site)
        bs = BeautifulSoup(url.download(cached = False)) 
        links = []
        for link in bs.findAll('a', href=True):
            links.append(link['href'])
    except:
        print("Can't get any links bruh...") 
    return links

def genre_finder(links, pattern):
    genres = [] 
    for link in links:
            if re.search(pattern, link):
                genres.append(link)
    return genres


links = getlinks(site) # get links from the page
#print(*links, sep="\n")

gpattern = 'https://podcasts.apple.com/in/genre/podcasts-' # find genres
genres = genre_finder(links, gpattern)
#print('Genres and sub-genres found :', len(genres), *genres, sep="\n")

ppattern = 'https://podcasts.apple.com/in/podcast/'
podcast_links = []

for genre in genres:
    all_links = getlinks(genre)
    podcast_links.append(genre_finder(all_links, ppattern))

podcast_links2 = list(set(np.concatenate(podcast_links).tolist()))


columns= ['Name', 'Artwork', 'Genre IDs', 'Episode Count', 'Episode Durations','iTunes URL', 'Feed URL', 'Podcast URL', 'Description']

df_podcast = pd.DataFrame([['-', '-', '-', '-', '-', '-', '-', '-', '-']], columns = columns)
df_podcast

,Name,Artwork,Genre IDs,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,-,-,-,-,-,-,-,-,-
